In [1]:
# %% Import libs
import numpy as np
import types
import sys,time
import h5py
import itertools
import imageio
import copy
import re

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from scipy.linalg import expm
from math import factorial
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import sph_harm
from matplotlib.colors import Normalize,LinearSegmentedColormap,LightSource
from matplotlib import cm
from matplotlib.ticker import LinearLocator



# %% Defining functions
# σ matrices

%matplotlib auto

I = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j], [1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
n = np.array([[0,0],[0,1]])

def kronecker_delta(x,y):
    if abs(x-y)<1e-5: return 1
    else: return 0



# Defining Physical constants
hbar = 6.62607015*(10**(-34))/2/np.pi   # Yet I'd like to set hbar = 1 in the calculation follows
# Defining Atomic Constants
Gamma = 2*np.pi*6*10**6     # 6MHz linewidth of the ^{87}Rb upper state 5P3/2
kappa = 2*np.pi*0.1*10**6   # 0.1MHz linewidth of the cavity
Omega_a = 2*np.pi*6.834*10**9 # 6.83Ghz energy gap for up and down state

def husimi_q(state, theta, phi):
    q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
    for j, t in enumerate(theta):
        for k, p in enumerate(phi):
            alpha = np.cos(t / 2) * np.exp(1j * p / 2)
            beta = np.sin(t / 2) * np.exp(-1j * p / 2)
            coherent_state = np.array([alpha ** m * beta ** (atom_number - m) * np.sqrt(factorial(atom_number) / (factorial(m) * factorial(atom_number - m))) for m in range(atom_number + 1)]).reshape(-1, 1)
            q_func[k, j] = np.abs(np.conj(coherent_state).T @ state) ** 2
    return np.abs(q_func)

# 专门开一个光子的类用来写多频率光的演化：
# typ表示这一个过程用Sx还是Sz
# order表示这个过程打光对应的频率
# delta是这个过程AC光的强度
class photons():
    def __init__(self,typ,order,delta,t):
        self.order = order
        delta[np.abs(delta)<1] = 0
        self.delta = delta*1.0
        self.t = t
        self.typ = typ

class sequence():
    def __init__(self,photons):
        self.photons = photons



class cavity_atom_light_system():   # Note that here most of the parameters are alterable, except for the atomic parameters.
    def __init__(self, atom_number, cooperativity, Delta, Omega):
        self.atom_number = atom_number              
        self.cooperativity = cooperativity              
        self.Delta = Delta                              # Delta should be much smaller than 6.8GHz/2, otherwise self.t() no longer holds
        self.omega_s = self.cooperativity*kappa*Gamma / (4*self.Delta)     # omega_s = g^2/Δ = ηκΓ/(4Δ)
        self.Omega = Omega                              # Note that we set Omega as a fixed parameter here.
        self.state_index = np.arange(self.atom_number+1)[::-1]    # The first one being the Dicke state with spin_z==+N/2
        # Calculating S_xyz, be ware of the order of Dicke states
        self.S_z = np.diag(np.array([self.atom_number/2 - iter for iter in np.arange(atom_number+1)]))
        S_plus  = np.array( [ [np.sqrt(atom_number/2 * (atom_number/2 +1) - jj * ( jj + 1 ) ) * kronecker_delta(ii, jj+1) for jj in np.arange(atom_number/2,-atom_number/2-1,-1)] for ii in np.arange(atom_number/2,-atom_number/2-1,-1)] )
        S_minus = np.array( [ [np.sqrt(atom_number/2 * (atom_number/2 +1) - jj * ( jj - 1 ) ) * kronecker_delta(ii, jj-1) for jj in np.arange(atom_number/2,-atom_number/2-1,-1)] for ii in np.arange(atom_number/2,-atom_number/2-1,-1)] )
        self.S_x = (S_plus + S_minus) / 2
        self.S_y = (S_plus - S_minus) / (2j)

        self.psi0 = self.initialize_psi0()
        self.psi = deepcopy(self.psi0)  # Remember to reset psi after each evolution turn

    def initialize_psi0(self):      # Initialize a CSS orienting at the north pole, that is, [1,0,0,0,...,0] in the Dicke representation
        psi0 = np.zeros(self.atom_number+1)
        psi0[0] = 1
        return psi0.reshape(-1,1)     # State vectors are column vectors, of course
    
    def H_AC(self, delta, order): # "delta" is the depth of the AC stark shift, and "order" is the Dicke state order with which the AC light resonates
        H_prime_exp = delta*(1 - 1j*Gamma / (2*self.Delta) ) * np.diag([n*np.abs(self.t(order*self.omega_s, n))**2 for n in range(self.atom_number+1)[::-1]]) / (order * np.abs(self.t(order * self.omega_s, order))**2 )   # Check that the first one being +N/2
        return H_prime_exp  # order = 1 recovers the Eq.6 in the main text of Creation of Greenberger-Horne-Zeilinger states with thousands of atoms by entanglement amplification

    # 开一个看能级结构的函数：
    def show_H_AC_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        H_ACs = np.array([[0 for i in range(self.atom_number+1)]],dtype = float)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            H_ACs = np.concatenate((H_ACs,np.array([np.diagonal(H_AC)])))
        H_ACs = np.delete(H_ACs,0,0)
        return H_ACs
    
        
    # 自己开一个读取photons sequence的函数
    def evolve_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            if photons[i].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
            elif photons[i].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
            else:
                return "Error, wrong input for S type."
    
    def evolve_photons_t(self,sequence,t_view):
        self.system_reset()
        photons = sequence.photons
        sequence_length = len(photons)
        t0 = 0
        marker = 0
        for i in range(sequence_length):
            if t_view>t0:
                t0 += photons[i].t
                marker += 1
            else:
                break
        if marker != 0:
            for i in range(marker-1):
                H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
                for j in range(len(photons[i].delta)):
                    H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
                if photons[i].typ == "x":
                    self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
                elif photons[i].typ == "z":
                    self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
                else:
                    return "Error, wrong input for S type."
            H_AC = self.H_AC(photons[marker-1].delta[0], photons[i].order[0])*0
            t_pre = 0
            for j in range(marker-1):
                t_pre += photons[j].t
            for j in range(len(photons[marker-1].delta)):
                H_AC += self.H_AC(photons[marker-1].delta[j], photons[marker-1].order[j])
            if photons[marker-1].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*(t_view-t_pre))@ self.psi
            elif photons[marker-1].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*(t_view-t_pre))@ self.psi
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        #返回该时刻的facecolors
        return husimi_q(self.psi,theta,phi)
    
    
    
    def evolve_typeC_params(self,params):
        t_x1, t_z, t_x2, order, delta, t  = params
        self.psi = expm(-1j*self.Omega*self.S_x*t_x2)@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        # self.show_occupation()
    
    def evolve_typeC_params_t(self, params, t_view):
        # 首先是读取这次演化的参数，目前只以typeC的为例
        t_x1, t_z, t_x2, order, delta, t  = params
        # 然后根据时序计算当时的state
        # 这里有一个巨大的坑，之后考虑IL和PSN得换成密度矩阵，狗听完死了
        if t_view<0:
            psi = self.psi
        elif t_view<t:
            psi = expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t_view) @ self.psi
        elif t_view<t+t_x1:
            psi = expm(-1j*self.Omega*self.S_x*(t_view-t))@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        elif t_view<t+t_x1+t_z:
            psi = expm(-1j*self.Omega*self.S_z*(t_view-t-t_x1))@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        elif t_view<t+t_x1+t_z+t_x2:
            psi = expm(-1j*self.Omega*self.S_x*(t_view-t-t_x1-t_z))@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        else:
            psi = expm(-1j*self.Omega*self.S_x*t_x2)@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        #返回该时刻的facecolors
        return husimi_q(psi,theta,phi)

            
    def evaluation(self, mode = "SSS"):
        if mode == "GHZ":
            density_mat = self.psi@np.transpose(np.conj(self.psi))
            fidelity = 1/2 * np.abs(density_mat[0,0] + density_mat[self.atom_number, self.atom_number] + np.abs(density_mat[0, self.atom_number] + density_mat[self.atom_number, 0]) )
            return fidelity
        elif mode == "SSS": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.psi@np.transpose(np.conj(self.psi))
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance +1/4*np.sqrt(loss_atom_num)
            Squeezing_Parameter = self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        elif mode == "SSSN": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.psi@np.transpose(np.conj(self.psi))
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance
            Squeezing_Parameter = np.real(np.trace(rho))*self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        else:
            p = re.compile('(N\d*)*')
            m = p.match(mode)
            if not m==None:
                rho = self.psi@np.transpose(np.conj(self.psi))
                nums = np.array([],dtype = int)
                pnum = re.compile("\d\d*")
                endmark = 0
                counter = 0
                while endmark<(len(mode)-1):
                    mnum = pnum.search(mode,endmark)
                    num = mode[mnum.span()[0]:mnum.span()[1]]
                    nums = np.append(nums,int(num))
                    endmark = mnum.span()[1]
                    counter += 1
                fidelity = 0.0
                for i in range(counter):
                    for j in range(counter):
                        fidelity += np.real(rho[nums[i],nums[j]])/counter
                return 1-fidelity,0


    # Cavity amplitude transmission function t
    def t(self, ksi, n_up):   # ksi is the detuning between light and cavity mode ω-ω_c
        return 1 / ( 1 + (n_up*self.cooperativity) / (1+4*(self.Delta+ksi)**2/Gamma**2) + ((self.atom_number-n_up)*self.cooperativity) / (1+4*(-Omega_a+self.Delta+ksi)**2/Gamma**2) - 2*1j* ( ksi/kappa - n_up*self.cooperativity*(self.Delta+ksi)/Gamma / (1+4*(self.Delta+ksi)**2/Gamma**2) - (self.atom_number-n_up)*self.cooperativity*(-Omega_a+self.Delta+ksi)/Gamma / (1+4*(-Omega_a+self.Delta+ksi)**2/Gamma**2) ) )

    def show_occupation(self):
        plt.figure()
        plt.scatter(self.state_index-self.atom_number/2, np.abs(np.transpose(self.psi))**2)     # Occupation is the probability distribution, for sure
        plt.show()

    def system_reset(self):
        self.psi = deepcopy(self.psi0)

    def husimi_q(self, theta, phi):
        q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
        for j, t in enumerate(theta):
            for k, p in enumerate(phi):
                alpha = np.cos(t / 2) * np.exp(1j * p / 2)
                beta = np.sin(t / 2) * np.exp(-1j * p / 2)
                coherent_state = np.array([alpha ** m * beta ** (self.atom_number - m) * np.sqrt(factorial(self.atom_number) / (factorial(m) * factorial(self.atom_number - m))) for m in range(self.atom_number + 1)]).reshape(-1, 1)
                q_func[k, j] = np.abs(np.conj(coherent_state).T @ self.psi) ** 2
        return np.abs(q_func)
    
    def visualize(self):
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        theta_grid, phi_grid = np.meshgrid(theta, phi)
        q_func = self.husimi_q(theta, phi)
        x = np.sin(theta_grid) * np.cos(phi_grid)
        y = np.sin(theta_grid) * np.sin(phi_grid)
        z = -np.cos(theta_grid)
        # Normalize the q_func values for the colormap
        norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
        q_norm = norm(q_func)
        colors = np.zeros(x.shape + (3,))
        colors[..., 0] = np.cos(q_norm*np.pi)**2*0.9
        colors[..., 1] = (1-np.sin(q_norm*np.pi*3/2)**2)*0.9
        colors[..., 2] = (1-np.sin(q_norm*np.pi*1/2)**2)*0.9

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.RdPu(norm(q_func)), alpha=0.7, linewidth=0, antialiased=True,shade=False)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Husimi-Q Representation')
#         ax.set_aspect('equal')
        plt.axis('off')
        plt.show()
    
    
    def rgb(self,colors,corders,norm):
        count = np.zeros(norm.shape,dtype = int)
        for i in range(len(corders)-1):
            count[norm>corders[i+1]]+=1
        recolors = np.zeros(norm.shape+(3,))
        recolors[...,0] = colors[count,0]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,0]*(norm-corders[count])/(corders[count+1]-corders[count])
        recolors[...,1] = colors[count,1]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,1]*(norm-corders[count])/(corders[count+1]-corders[count])
        recolors[...,2] = colors[count,2]*(corders[count+1]-norm)/(corders[count+1]-corders[count]) +colors[count+1,2]*(norm-corders[count])/(corders[count+1]-corders[count])
        return recolors
    
    def visualize_withwall(self,diag):
        theta = np.linspace(0, np.pi, 200)
        phi = np.linspace(0, np.pi*4/2, 200)
        theta_grid, phi_grid = np.meshgrid(theta, phi)
        q_func = self.husimi_q(theta, phi)
        x = np.sin(theta_grid) * np.cos(phi_grid)
        y = np.sin(theta_grid) * np.sin(phi_grid)
        z = -np.cos(theta_grid)
        # Normalize the q_func values for the colormap
        norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
        q_norm = norm(q_func)
#         for i in range(200):
#             print(q_norm[i])
#         clsphi = np.array([[245,245,245],[237,90,179],[22,64,214],[0,27,121]])/256
        clsphi = np.array([[240,240,240],[232,218,184],[240,200,0],[230,100,0],[250,0,0]])/256
#         corderphi = np.array([0,0.05,0.2,0.5,1])
        corderphi = np.array([0,0.35,0.5,0.7,1])
        colors_phi = np.zeros(x.shape + (3,))
        colors_phi = self.rgb(clsphi,corderphi,q_norm)
#         colors_phi[..., 0] = np.cos(q_norm*np.pi)**2*0.9
#         colors_phi[..., 1] = (1-np.sin(q_norm*np.pi*3/2)**2)*0.9
#         colors_phi[..., 2] = (1-np.sin(q_norm*np.pi*2/2)**2)*0.9
        
        diagabs= np.abs(diag)
        wall = np.zeros(x.shape)
        for i in range(len(theta)):
            for j in range(len(phi)):
                rightwhere = (1-np.cos(np.pi-theta[i]))*self.atom_number/2*0.999999
                rightint = int(rightwhere)
                wall[j,i] = diagabs[rightint]*(rightint+1-rightwhere) + diagabs[rightint+1]*(rightwhere-rightint)
        norm_wall = Normalize(vmin=np.min(wall), vmax=np.max(wall))
        q_wall = norm_wall(wall)
        colors_wall = np.zeros(x.shape + (3,))
        colors_wall[..., 0] = 1-q_wall*0.4
        colors_wall[..., 1] = 1-q_wall*0.4
        colors_wall[..., 2] = 1
        
        colors = np.zeros(x.shape + (3,))
        colors[..., 0] = np.minimum(colors_wall[...,0],colors_phi[...,0])
        colors[..., 1] = np.minimum(colors_wall[...,1],colors_phi[...,1])
        colors[..., 2] = np.minimum(colors_wall[...,2],colors_phi[...,2])

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=colors, alpha=0.7, linewidth=0, antialiased=True,shade=False)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Husimi-Q Representation')
        plt.axis('off')
        plt.show()
        

        
# 2023/08/04更新，我们需要先找到一个最好的H_AC才行，然后用固定协同率的光子去找最合适的参数
# 为此我们专门开一个找H_AC的函数：
# 当然这里我们只关心实部是多少
def find_H_AC(atom_number,ini_diag = np.array([None,None]),ini_t = np.array([None,None]),times = 1000):
    T0 = 0.003
    T = T0
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 100, Delta = -2*np.pi*4*10**9, Omega = 0.2*10**6*2*np.pi)
    if (ini_diag == None).all():
        ini_diag = -np.ones(atom_number+1,dtype = complex)*10**6
    if (ini_t == None).all():
        ini_t = np.array([1.0*10**(-6),0.6*10**(-6)])
    system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
    system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*ini_t[1])@ system.psi
    ini_Xr = system.evaluation(mode = "SSSN")[0]
    grad_len_ratio = 1.0/times
    step_len_ratio = 10.0/times
    for time in range(times):
        dXr = 0
        for order in range(atom_number+1):
            new_dXr = 0
            direction = np.random.randint(2)
            system.system_reset()
            grad_diag = ini_diag*1.0
            grad_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))] = grad_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+grad_len_ratio)
            system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(grad_diag))*ini_t[1])@ system.psi
            grad_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            sign = -np.sign(grad_Xr-ini_Xr)
            new_diag = ini_diag*1.0
            new_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))] = new_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+step_len_ratio*sign*np.random.rand())
            system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(new_diag))*ini_t[1])@ system.psi
            new_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            if new_Xr<ini_Xr:
                T = T/(1+100/times/atom_number)
                new_dXr = ini_Xr-new_Xr
                ini_Xr = deepcopy(new_Xr)
                ini_diag = new_diag*1.0
            else:
                Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/atom_number)
                    ini_Xr = deepcopy(new_Xr)
                    ini_diag = new_diag*1.0
            if new_dXr>dXr:
                dXr = new_dXr*1.0
            t_random = np.random.randint(2)
            grad_t = ini_t*1.0
            grad_t[t_random]*=(1+grad_len_ratio)
            system.psi = expm(-1j*(system.Omega*system.S_x)*grad_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*grad_t[1])@ system.psi
            grad_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            sign = -np.sign(grad_Xr-ini_Xr)
            new_t = ini_t*1.0
            new_t[t_random]*=(1+step_len_ratio*sign*np.random.rand())
            system.psi = expm(-1j*(system.Omega*system.S_x)*new_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*new_t[1])@ system.psi
            new_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            if new_Xr<ini_Xr:
                T = T/(1+100/times/atom_number)
                new_dXr = ini_Xr-new_Xr
                ini_Xr = deepcopy(new_Xr)
                ini_t = new_t*1.0
            else:
                Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/atom_number)
                    ini_Xr = deepcopy(new_Xr)
                    ini_t = new_t*1.0
            if new_dXr>dXr:
                dXr = new_dXr*1.0
            print("\r", end="")
            print(f"Scan finished:{time+1}/{times}, Xr = {ini_Xr}, order = {order}, dXr = {dXr}", end="")
            sys.stdout.flush()
        if dXr < ini_Xr*10**(-5):
            T = T*(1+25/atom_number)
    return ini_diag,ini_t


def find_H_AC_supper(atom_number,target,ini_diag = np.array([None,None]),ini_t = np.array([None,None]),ini_typ = np.array([None,None]),times = 1000,loss = False,eta = 100):
    T0 = 0.003
    T = T0
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = eta, Delta = -2*np.pi*3.4*10**9, Omega = 0.2*10**6*2*np.pi)
    if loss:
        losscoe = (1 - 1j*Gamma / (2*system.Delta) )
    else:
        losscoe = 1
    if (ini_diag == None).all():
        ini_diag = np.array([-np.ones(atom_number+1,dtype = complex)*10**6,-np.ones(atom_number+1,dtype = complex)*10**6])
    if (ini_t == None).all():
        ini_t = np.array([1.0*10**(-6),0.6*10**(-6)])
    if (ini_typ == None).all():
        ini_typ = np.array(["x","x"])
        
    for diag in range(len(ini_diag)):
        if ini_typ[diag] == "x":
            rotation = system.S_x
        elif ini_typ[diag] == "y":
            rotation = system.S_y
        elif ini_typ[diag] == "z":
            rotation = system.S_z
        system.psi = expm(-1j*losscoe*(system.Omega*rotation + np.diag(ini_diag[diag]))*ini_t[diag])@ system.psi
    ini_Xr = system.evaluation(mode = target)[0]
    grad_len_ratio = 1.0/times
    step_len_ratio = 10.0/times
    for time in range(times):
        dXr = 0
        for Diag in range(len(ini_diag)):
            for order in range(atom_number+1):
                new_dXr = 0
                direction = np.random.randint(2)
                system.system_reset()
                grad_diag = ini_diag*1.0
                grad_diag[Diag,int(np.mod(order*((-1)**(direction)),atom_number+1))] = grad_diag[Diag,int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+grad_len_ratio)
                for diag in range(len(ini_diag)):
                    if ini_typ[diag] == "x":
                        rotation = system.S_x
                    elif ini_typ[diag] == "y":
                        rotation = system.S_y
                    elif ini_typ[diag] == "z":
                        rotation = system.S_z
                    system.psi = expm(-1j*losscoe*(system.Omega*rotation + np.diag(grad_diag[diag]))*ini_t[diag])@ system.psi
                grad_Xr = system.evaluation(mode = target)[0]
                system.system_reset()
                sign = -np.sign(grad_Xr-ini_Xr)
                new_diag = ini_diag*1.0
                new_diag[Diag,int(np.mod(order*((-1)**(direction)),atom_number+1))] = new_diag[Diag,int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+step_len_ratio*sign*np.random.rand())
                for diag in range(len(ini_diag)):
                    if ini_typ[diag] == "x":
                        rotation = system.S_x
                    elif ini_typ[diag] == "y":
                        rotation = system.S_y
                    elif ini_typ[diag] == "z":
                        rotation = system.S_z
                    system.psi = expm(-1j*losscoe*(system.Omega*rotation + np.diag(new_diag[diag]))*ini_t[diag])@ system.psi
                new_Xr = system.evaluation(mode = target)[0]
                system.system_reset()
                if new_Xr<ini_Xr:
                    T = T/(1+100/times/atom_number)
                    new_dXr = ini_Xr-new_Xr
                    ini_Xr = deepcopy(new_Xr)
                    ini_diag = new_diag*1.0
                else:
                    Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        T = T/(1+100/times/atom_number)
                        ini_Xr = deepcopy(new_Xr)
                        ini_diag = new_diag*1.0
                if new_dXr>dXr:
                    dXr = new_dXr*1.0
                t_random = np.random.randint(len(ini_diag))
                grad_t = ini_t*1.0
                grad_t[t_random]*=(1+grad_len_ratio)
                for diag in range(len(ini_diag)):
                    if ini_typ[diag] == "x":
                        rotation = system.S_x
                    elif ini_typ[diag] == "y":
                        rotation = system.S_y
                    elif ini_typ[diag] == "z":
                        rotation = system.S_z
                    system.psi = expm(-1j*losscoe*(system.Omega*rotation + np.diag(ini_diag[diag]))*grad_t[diag])@ system.psi
                grad_Xr = system.evaluation(mode = target)[0]
                system.system_reset()
                sign = -np.sign(grad_Xr-ini_Xr)
                new_t = ini_t*1.0
                new_t[t_random]*=(1+step_len_ratio*sign*np.random.rand())
                for diag in range(len(ini_diag)):
                    if ini_typ[diag] == "x":
                        rotation = system.S_x
                    elif ini_typ[diag] == "y":
                        rotation = system.S_y
                    elif ini_typ[diag] == "z":
                        rotation = system.S_z
                    system.psi = expm(-1j*losscoe*(system.Omega*rotation + np.diag(ini_diag[diag]))*new_t[diag])@ system.psi
                new_Xr = system.evaluation(mode = target)[0]
                system.system_reset()
                if new_Xr<ini_Xr:
                    T = T/(1+100/times/atom_number)
                    new_dXr = ini_Xr-new_Xr
                    ini_Xr = deepcopy(new_Xr)
                    ini_t = new_t*1.0
                else:
                    Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        T = T/(1+100/times/atom_number)
                        ini_Xr = deepcopy(new_Xr)
                        ini_t = new_t*1.0
                if new_dXr>dXr:
                    dXr = new_dXr*1.0
                print("\r", end="")
                print(f"Scan finished:{time+1}/{times}, Xr = {round(ini_Xr,9)}, Diag = {Diag},order = {order}, dXr = {round(dXr,9)}", end="")
                sys.stdout.flush()
        if dXr < ini_Xr*10**(-5):
            T = T*(1+25/atom_number)
    return ini_diag,ini_t

# 整个遗传优化算法试试看
def genetic_evolution(generations,size,cooperativity, atom_number, ini_Delta, target, inisequence = None,seq_evol = 0,pho_evol = 1,Delta_evol = 1):
    # 每一代的数量
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = cooperativity, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if inisequence == None:
        old_generation = np.array([sequence([photons("x",np.array([atom_number/2]),np.array([-100*10**6]),1.0*10**(-6))]) for i in range(100)])
    
    else:
        old_generation = np.array([inisequence for i in range(size)])
    old_Delta = np.array([ini_Delta for i in range(size)])
    # 第一代
    
    old_behavior = np.ones(size,dtype = float)
    old_phnum = np.array([])
    old_seqnum = np.array([])
    for i in range(size):
        system.Delta = old_Delta[i]
        system.psi = deepcopy(system.psi0)
        system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
        system.evolve_photons(old_generation[i])
        Xr, Jabs, polartheta, polorphi = system.evaluation(mode = target)
        old_behavior[i] = Xr
        old_seqnum = np.append(old_seqnum,len(old_generation[i].photons))
        for pho in range(len(old_generation[i].photons)):
            old_phnum = np.append(old_phnum,len(old_generation[i].photons[pho].delta))
    for evolve in range(generations):
        # 空的新一代
        new_generation = np.array([])
        # 每次产生新的一代需要有一定概率增加photon的个数，并且这玩意儿应该有个保底：
        if seq_evol == 1:
            Ps = 0.001
        else:
            Ps = 0
        if pho_evol ==1:
            Pp = 0.01
        else:
            Pp = 0
        step_len_ratio = 10**(-2)
        if Delta_evol == 1:
            new_Delta = old_Delta*((1-2*np.random.rand(size))*step_len_ratio+1)
        else:
            new_Delta = old_Delta*1.0
        new_behavior = np.ones(size,dtype = float)
        for i in range(size):
            news_judge = np.random.rand()
            newp_judge = np.random.rand()
            # 先继承并在原本的数值上稍微动一点
            new_photons = np.array([])
            for seq in range(len(old_generation[i].photons)):
                l = len(old_generation[i].photons[seq].delta)
                delta = old_generation[i].photons[seq].delta*((1-2*np.random.rand(l))*step_len_ratio+1)
                order = old_generation[i].photons[seq].order*((1-2*np.random.rand(l))*step_len_ratio+1)
                if newp_judge<Pp:
                    delta = np.append(delta,delta[-1])*np.random.rand()*step_len_ratio
                    order = np.append(order,order[-1]*((5*np.random.rand())*step_len_ratio+1))
                order_sort = np.argsort(order)
                delete_ind = np.array([],dtype = int)
                for ind in range(len(order_sort)-1):
                    if order[order_sort[ind+1]]-order[order_sort[ind]]<atom_number/1000:
                        delete_ind = np.append(delete_ind,int(order_sort[ind+1]))
                        delta[order_sort[ind]] = delta[order_sort[ind]]+delta[order_sort[ind+1]]
                order = np.delete(order,delete_ind)
                delta = np.delete(delta,delete_ind)
                delta[np.abs(delta)<10**(6)] = 0
                t = old_generation[i].photons[seq].t*((1-2*np.random.rand())*step_len_ratio+1)
                typ = old_generation[i].photons[seq].typ
                new_photons = np.append(new_photons, photons(typ,order,delta,t))
            if news_judge>Ps:
                Ps = Ps*1.1
            else:
                seq = -1
                l = len(old_generation[i].photons[seq].delta)
                delta = old_generation[i].photons[seq].delta*((1-2*np.random.rand(l))*step_len_ratio+1)*step_len_ratio
                order = old_generation[i].photons[seq].order*((1-2*np.random.rand(l))*step_len_ratio+1)
                t = old_generation[i].photons[seq].t*((1-2*np.random.rand())*step_len_ratio+1)*step_len_ratio
                randtyp = np.random.randint(2)
                if randtyp == 1:
                    typ = "x"
                else:
                    typ = "z"
                    
                order_sort = np.argsort(order)
                delete_ind = np.array([],dtype = int)
                for ind in range(len(order_sort)-1):
                    if order[order_sort[ind+1]]-order[order_sort[ind]]<atom_number/1000:
                        delete_ind = np.append(delete_ind,int(order_sort[ind+1]))
                        delta[order_sort[ind]] = delta[order_sort[ind]]+delta[order_sort[ind+1]]
                order = np.delete(order,delete_ind)
                delta = np.delete(delta,delete_ind)
                new_photons = np.append(new_photons, photons(typ,order,delta,t))
                Ps = 0.001
            seq_delete_ind = np.array([],dtype = int)
            for seq in range(len(new_photons)-1):
                order_judge1 = (len(new_photons[seq].order) == len(new_photons[seq+1].order))
                delta_judge = False
                order_judge = False
                if order_judge1:
                    sort1 = np.argsort(new_photons[seq].order)
                    sort2 = np.argsort(new_photons[seq+1].order)
                    order_judge = (new_photons[seq].order[sort1] - new_photons[seq+1].order[sort2] < atom_number*1/500/len(new_photons[seq].order)).all()
                    delta_judge = (new_photons[seq].delta[sort1] - new_photons[seq+1].delta[sort2] < 1.0*10**6/len(new_photons[seq].order)).all()
                if delta_judge and order_judge:
                    seq_delete_ind = np.append(seq_delete_ind,seq)
                    new_photons[seq+1].t = new_photons[seq+1].t+new_photons[seq].t
            new_photons = np.delete(new_photons,seq_delete_ind)
            
            new_generation = np.append(new_generation,sequence(new_photons))
            system.Delta = new_Delta[i]
            system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
            system.psi = deepcopy(system.psi0)
            system.evolve_photons(new_generation[i])
            Xr, Jabs, polartheta, polorphi = system.evaluation(mode = target)
            new_behavior[i] = Xr
        # 2023/07/27更新代码验证完毕，开始进化步骤：
        total_generation = np.concatenate((old_generation,new_generation))
        total_Delta = np.concatenate((old_Delta,new_Delta))
        total_behavior = np.concatenate((old_behavior,new_behavior))
        # 注意np.argsort从小到大排序
        total_rank = np.argsort(total_behavior)
        for i in range(size):
            old_generation[i] = total_generation[total_rank[i]]
            old_Delta[i] = total_Delta[total_rank[i]]
            old_behavior[i] = total_behavior[total_rank[i]]
        old_phnum = np.array([])
        old_seqnum = np.array([])
        for i in range(size):
            old_seqnum = np.append(old_seqnum,len(old_generation[i].photons))
            for pho in range(len(old_generation[i].photons)):
                old_phnum = np.append(old_phnum,len(old_generation[i].photons[pho].delta))
        max_phnum = np.max(old_phnum)
        max_seqnum = np.max(old_seqnum)
        Xrbest = np.min(old_behavior)
        print("\r", end="")
        print(f"Evolution finished:{evolve}/{generations},max phnum = {max_phnum}, max seqnum = {max_seqnum} best Xr = {Xrbest}", end="")
        sys.stdout.flush()
    all_sort = np.argsort(old_behavior)
    return old_generation[all_sort[0]],old_Delta[all_sort[0]],old_behavior[all_sort[0]]

def multi_frequency_scan(etamax,atom_number, target,ini_Delta = None, inisequence = None, times = 1000, opt_Delta = False):
    T = 0.0003
    if ini_Delta == None:
        ini_Delta =  -4115000000.0
    ini_eta = etamax
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = ini_eta, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if inisequence == None:
        ini_sequence = sequence([photons("x",np.array([atom_number/2]),np.array([-100*10**6]),1.0*10**(-6))])
    else:
        ini_sequence = inisequence
        
    ini_photons = inisequence.photons
    nseq = len(ini_photons)
    
    system.evolve_photons(ini_sequence)
    res = system.evaluation(mode = target)[0]
    min_res = res*1.0
    system.system_reset()
    
    grad_len_ratio = 10.0/times
    step_len_ratio = 40.0/times
    
    for time in range(times):
        dres = 0
        for seq in range(nseq):
            ini_order = ini_photons[seq].order
            ini_delta = ini_photons[seq].delta
            ini_t = ini_photons[seq].t
            new_dres = 0
            for nm in range(len(ini_photons[seq].order)):
                new_order = ini_order*1.0
                grad_order = ini_order*1.0
                grad_delta = ini_delta*1.0
                grad_order[nm] = grad_order[nm]*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",grad_order,grad_delta,ini_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_order[nm] = new_order[nm]*(1+step_len_ratio*sign*np.random.rand())
                new_delta = ini_delta*1.0
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",new_order,new_delta,ini_t))
                new_sequence = sequence(new_photons)

                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_order = new_order*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_order = new_order*1.0
                if new_dres>dres:
                    dres = new_dres*1.0

                new_order = ini_order*1.0
                grad_order = ini_order*1.0
                grad_delta = ini_delta*1.0
                grad_delta[nm] = grad_delta[nm]*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",grad_order,grad_delta,ini_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_delta[nm] = new_delta[nm]*(1+step_len_ratio*sign*np.random.rand())
                new_order = ini_order*1.0
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",new_order,new_delta,ini_t))
                new_sequence = sequence(new_photons)


                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_delta = new_delta*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_delta = new_delta*1.0
                
                grad_t = ini_t*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",ini_order,ini_delta,grad_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_t = ini_t*(1+step_len_ratio*sign*np.random.rand())
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",ini_order,ini_delta,new_t))
                new_sequence = sequence(new_photons)


                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_t = new_t*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_t = new_t*1.0
                
                if opt_Delta:
                    Delta_ini_photons = np.array([],dtype = object)
                    for ns in range(nseq):
                        if not ns == seq:
                            Delta_ini_photons = np.append(Delta_ini_photons,ini_photons[ns])
                        else:
                            Delta_ini_photons = np.append(Delta_ini_photons,photons("x",ini_order,ini_delta,ini_t))
                    Delta_ini_sequence = sequence(grad_photons)

                    grad_Delta = ini_Delta*(1+grad_len_ratio)

                    system.Delta = grad_Delta
                    system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)

                    system.evolve_photons(Delta_ini_sequence)
                    grad_res = system.evaluation(mode = target)[0]
                    system.system_reset()

                    sign = -np.sign(grad_res-res)
                    new_Delta = ini_Delta*(1+step_len_ratio*sign*np.random.rand())

                    system.Delta = new_Delta
                    system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
                    system.evolve_photons(Delta_ini_sequence)
                    new_res = system.evaluation(mode = target)[0]
                    system.system_reset()

                    if new_res<res:
                        T = T/(1+500/times/atom_number)
                        new_dres = res-new_res
                        res = deepcopy(new_res)
                        if res<min_res:
                            min_res = res*1.0
                        ini_Delta = new_Delta*1.0
                    else:
                        Tjudge = np.exp(-(new_res-res)/T)
                        random = np.random.rand()
                        if random<Tjudge:
                            res = deepcopy(new_res)
                            ini_Delta = new_Delta*1.0
                        else:
                            system.Delta = ini_Delta
                            system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
                
                if new_dres>dres:
                    dres = new_dres*1.0

                print("\r", end="")
                print(f"Scan finished:{time+1}/{times},min_res = {min_res},Delta = {ini_Delta},res = {res}, dres = {dres}", end="")
                sys.stdout.flush()
            ini_photons[seq] = photons(ini_photons[seq].typ,ini_order,ini_delta,ini_t)
        if dres < res*10**(-5):
            T = T*(1+7.5/atom_number)
            
    final = sequence(ini_photons)
    for i in range(len(final.photons)):
        print(f"delta{i} = ",final.photons[i].delta)
        print(f"order{i} = ",final.photons[i].order)
        print(f"t{i} = ",final.photons[i].t)
        print(f"typ{i} = \""+final.photons[i].typ+"\"")
    print("Delta = ",ini_Delta)
    
    return final,ini_Delta

# 再用有限个频率的光拟合特定的能级结构
# 需要考虑的是墙后面的部分(原子态到不了的地方)不重要,所以权重需要调整一下:
def fit_energy_level(target,Np,etamax,ini_order = np.array([None,None]),ini_delta = np.array([None,None]), times = 1000):
    N = len(target)-1
    T = 0.3
    ini_Delta =  -4115000000.0
    ini_eta = etamax
    system = cavity_atom_light_system(atom_number = N, cooperativity = ini_eta, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if (ini_order == None).all():
        ini_order = np.array([N/3+i*N/3/Np for i in range(Np)])
    if (ini_delta == None).all():
        ini_delta = np.array([np.min(target)/(i+1)/(N+1)*(1+ini_order[i]*etamax/(1+4*(ini_Delta+ini_order[i]*system.omega_s)**2/Gamma**2)) for i in range(Np)])
    ini_sequence = sequence([photons("x",ini_order,ini_delta,1.0e-6)])
    res = np.sum(((target-np.real(system.show_H_AC_photons(ini_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
    
    fig,ax = plt.subplots()
    ax.scatter(np.arange(N+1),target)
    ax.scatter(np.arange(N+1),np.real(system.show_H_AC_photons(ini_sequence)[0]))
    plt.show()
    
    grad_len_ratio = 10.0/times
    step_len_ratio = 100.0/times
    for time in range(times):
        dres = 0
        for order in range(Np):
            new_dres = 0
            
            new_order = ini_order*1.0
            grad_order = ini_order*1.0
            grad_delta = ini_delta*1.0
            grad_order[order] = grad_order[order]*(1+grad_len_ratio)
            grad_sequence = sequence([photons("x",grad_order,grad_delta,1.0e-6)])
            grad_res = np.sum(((target-np.real(system.show_H_AC_photons(grad_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            sign = -np.sign(grad_res-res)
            new_order[order] = new_order[order]*(1+step_len_ratio*sign*np.random.rand())
            new_delta = ini_delta*1.0
            new_sequence = sequence([photons("x",new_order,new_delta,1.0e-6)])
            new_res = np.sum(((target-np.real(system.show_H_AC_photons(new_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            
            if new_res<res:
                T = T/(1+100/times/N)
                new_dres = res-new_res
                res = deepcopy(new_res)
                ini_order = new_order*1.0
            else:
                Tjudge = np.exp(-(new_res-res)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/N)
                    res = deepcopy(new_res)
                    ini_order = new_order*1.0
            if new_dres>dres:
                dres = new_dres*1.0
                
            new_order = ini_order*1.0
            grad_order = ini_order*1.0
            grad_delta = ini_delta*1.0
            grad_delta[order] = grad_delta[order]*(1+grad_len_ratio)
            grad_sequence = sequence([photons("x",grad_order,grad_delta,1.0e-6)])
            grad_res = np.sum(((target-np.real(system.show_H_AC_photons(grad_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            sign = -np.sign(grad_res-res)
            new_delta[order] = new_delta[order]*(1+step_len_ratio*sign*np.random.rand())
            new_order = ini_order*1.0
            new_sequence = sequence([photons("x",new_order,new_delta,1.0e-6)])
            new_res = np.sum(((target-np.real(system.show_H_AC_photons(new_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            
            if new_res<res:
                T = T/(1+100/times/N)
                new_dres = res-new_res
                res = deepcopy(new_res)
                ini_delta = new_delta*1.0
            else:
                Tjudge = np.exp(-(new_res-res)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/N)
                    res = deepcopy(new_res)
                    ini_delta = new_delta*1.0
            if new_dres>dres:
                dres = new_dres*1.0
                
            print("\r", end="")
            print(f"Scan finished:{time+1}/{times}, res = {res}, dres = {dres}", end="")
            sys.stdout.flush()
        if dres < res*10**(-5):
            T = T*(1+25/N)
    
    ini_sequence = sequence([photons("x",ini_order,ini_delta,1.0e-6)])
    fig,ax = plt.subplots()
    ax.scatter(np.arange(N+1),target)
    ax.scatter(np.arange(N+1),np.real(system.show_H_AC_photons(ini_sequence)))
    plt.show()
    
    return photons("x",ini_order,ini_delta,1.0e-6)
    
        
# 搞动图的，默认duration是0.05s
def create_gif(image_list, gif_name, duration=0.05):
    frames = []
    # 第一张图停1s
    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[0]))
    # 中间的图
    for image_name in image_list:
        frames.append(imageio.imread(image_name))
    # 最后一张图停1s
    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[-1]))
    imageio.mimsave(gif_name, frames, 'GIF', duration=duration)
    return None

Using matplotlib backend: Qt5Agg


In [5]:
# |GHZ>
atom_number = 50
target = "N0N50"
ini_diag = np.array([[-1.56342088e+06-0.j, -1.53145752e+06-0.j, 
-1.29979231e+06-0.j, -8.60319359e+05-0.j, -5.35801998e+05-0.j, 
-5.52692444e+05-0.j, -6.07998001e+05-0.j, -5.08885712e+05-0.j, 
-5.75110646e+05-0.j, -6.88404594e+05-0.j, -4.49372982e+05-0.j, 
-3.31430986e+05-0.j, -4.12112181e+05-0.j, -2.08363114e+05-0.j, 
-4.28029904e+05-0.j, -8.28937700e+05-0.j, -5.23711039e+05-0.j, 
-9.16539148e+05-0.j, -1.46536002e+06-0.j, -7.83463038e+05-0.j, 
-1.49859851e+06-0.j, -1.56647523e+06-0.j, -5.39112804e+05-0.j, 
-5.18691463e+06-0.j, -5.29732202e+05-0.j, -2.14604316e+07-0.j, 
-9.62979812e+05-0.j, -8.82475720e+06-0.j, -7.04812442e+06-0.j, 
-3.25421638e+05-0.j, -7.42995310e+05-0.j, -3.06470481e+06-0.j, 
-3.00317469e+06-0.j, -2.33649118e+06-0.j, -1.82463793e+06-0.j, 
-1.94969226e+06-0.j, -2.61427987e+06-0.j, -3.07200991e+06-0.j, 
-3.15250586e+06-0.j, -3.02677721e+06-0.j, -2.78410176e+06-0.j, 
-2.48804940e+06-0.j, -2.33105438e+06-0.j, -2.40095696e+06-0.j, 
-2.41623428e+06-0.j, -2.21041303e+06-0.j, -1.85118413e+06-0.j, 
-1.51940903e+06-0.j, -1.35396918e+06-0.j, -1.35229146e+06-0.j, -1.40954249e+06-0.j],[0 for i in range(51)]])
for i in range(51):
    if np.abs(i-25)>5:
        ini_diag[0,i] = 0
    else:
        ini_diag[0,i] = -1.7e07*1/(1+(i-25)**2/2**2)
ini_t = np.array([2.5e-6,0.0])
ini_typ = np.array([],dtype = str)
for i in range(2):
    if np.mod(i,3) == 0:
        ini_typ = np.append(ini_typ,"x")
    elif np.mod(i,3) == 1:
        ini_typ = np.append(ini_typ,"x")
    else:
        ini_typ = np.append(ini_typ,"x")
ini_diag,ini_t = find_H_AC_supper(atom_number,target,ini_diag = ini_diag,ini_t = ini_t,ini_typ = ini_typ,times = 50)
ini_diag,ini_t = find_H_AC_supper(atom_number,target,ini_diag = ini_diag,ini_t = ini_t,ini_typ = ini_typ,times = 50)
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 100, Delta = -2*np.pi*3.4*10**9, Omega = 0.2*10**6*2*np.pi)

for diag in range(len(ini_diag)):
    if ini_typ[diag] == "x":
        rotation = system.S_x
    elif ini_typ[diag] == "y":
        rotation = system.S_y
    elif ini_typ[diag] == "z":
        rotation = system.S_z
    system.psi = expm(-1j*(system.Omega*rotation + np.diag(ini_diag[diag]))*ini_t[diag])@ system.psi

system.visualize_withwall(ini_diag[0])
Xr = system.evaluation(mode = target)[0]
print(Xr)

delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
t0 =  4.1399943765870936e-07
typ0 = "x"
delta1 =  np.array([-1770840.01367247])
order1 =  np.array([32.74001811149749])
t1 =  2.1773138739050565e-06
typ1 = "x"
Delta =  -2*np.pi*3.4*10**9

ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

atom_number = 50

system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
system.evolve_photons(ini_sequence)
diag = system.show_H_AC_photons(ini_sequence)[1]
# print(np.abs(diag))
# fig,ax = plt.subplots()
# ax.scatter(np.arange(atom_number+1),np.abs(diag))
# plt.show()
system.visualize_withwall(diag)

# system.visualize()

Scan finished:50/50, Xr = 0.00543, Diag = 1,order = 50, dXr = 1.875e-06-060580.005430000175174476


In [3]:
print("GHZnfinf = ",list(np.real(ini_diag[0])))
print("t = ",list(ini_t))

GHZnfinf =  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1404111.2606374905, -386.53947974471083, -790579.2929180233, -10546261.986390863, -4129877.7077810736, -22502690.07331861, -22053085.74786825, -1390071.2506806278, -1010329.3513136582, -5866633.137376489, -5133410.941661359, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t =  [2.495608664005765e-06, 0.0]


In [9]:
# |GHZ> Steps
delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
t0 =  4.1399943765870936e-07
typ0 = "x"
delta1 =  np.array([-1770840.01367247])
order1 =  np.array([32.74001811149749])
t1 =  2.1773138739050565e-06
typ1 = "x"
Delta =  -2*np.pi*3.4*10**9

ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

atom_number = 50

system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
diag = system.show_H_AC_photons(ini_sequence)[1]
system.visualize_withwall(diag)
system.evolve_photons_t(ini_sequence,t0+t1*(1/2-0.07))
system.visualize_withwall(diag)
system.evolve_photons_t(ini_sequence,t0+t1*(1/2-0.02))
system.visualize_withwall(diag)
system.evolve_photons_t(ini_sequence,t0+t1)
system.visualize_withwall(diag)
print(list(np.real(diag)))

[-308584.1206891501, -327139.1309327958, -347776.13564744074, -370838.69910094794, -396746.26701159024, -426015.4851376772, -459288.8982607367, -497374.1110902036, -541298.0208685438, -592383.1499896534, -652357.0199570308, -723511.9944502125, -808944.062753395, -912918.4006092567, -1041444.6371032124, -1203210.7071064857, -1411153.2697051964, -1685207.145867618, -2057346.0795741724, -2581324.5987334466, -3352659.5966656697, -4552330.00975591, -6548067.098107626, -10130418.088048229, -16906473.90015136, -27965239.090602268, -32718762.482091762, -21769093.876088895, -11808745.678059278, -6660385.22799113, -4059777.676417652, -2645042.998225556, -1813861.4285509815, -1293033.72090182, -949293.8700940796, -712746.4060287338, -544333.3006452653, -421022.88452856644, -328602.468070306, -257956.4331254642, -203042.0047600006, -159737.63750635312, -125161.64150172408, -97254.95510585656, -74517.82276734484, -55839.09644529411, -40382.908899400245, -27511.81103711445, -16733.627810945163, -766

In [11]:
#|Lantern>
target = "N10N20N30N40"
ini_diag = np.array([[-3.69419234e+06+0.j, -3.22893640e+06+0.j, -5.00058320e+06+0.j,
       -2.79841633e+06+0.j, -2.36476169e+06+0.j, -8.86845772e+06+0.j,
       -1.16220639e+07+0.j, -7.07174941e+06+0.j, -2.91275808e-05+0.j,
       -7.87237702e+06+0.j, -1.79739730e+07+0.j, -2.01607004e+07+0.j,
       -2.05188026e+07+0.j, -1.56821854e+07+0.j, -1.09035967e+07+0.j,
       -1.59218550e+07+0.j, -2.20960351e+07+0.j, -1.16042221e+07+0.j,
       -6.89418543e+06+0.j, -1.82046883e+07+0.j, -2.77978262e-02+0.j,
       -4.37514085e+05+0.j, -2.02425786e+06+0.j, -7.56609966e+06+0.j,
       -2.30834440e+07+0.j, -1.31294537e+06+0.j, -5.39459229e+06+0.j,
       -2.58254283e+00+0.j, -1.60340133e+06+0.j, -7.09182453e+06+0.j,
       -2.60091925e+07+0.j, -2.28615718e+07+0.j, -9.57053560e+06+0.j,
       -1.27230231e+07+0.j, -3.19693008e+07+0.j, -1.61172825e+07+0.j,
       -1.34084411e+08+0.j, -2.45740907e+07+0.j, -1.57830894e+07+0.j,
       -6.14677630e-05+0.j, -1.10471061e+00+0.j, -3.39751388e+03+0.j,
       -7.05990219e+06+0.j, -6.99015484e-02+0.j, -1.39477361e+07+0.j,
       -2.34833249e-02+0.j, -2.54078916e-01+0.j, -4.12100584e+06+0.j,
       -1.19800925e-01+0.j, -1.36555525e+07+0.j, -1.12054889e+01+0.j],
 [-2.45192662e+03+0.j, -4.17290124e+07+0.j, -2.98480686e+07+0.j,
       -9.28757468e-02+0.j, -4.17278721e+07+0.j, -2.22122403e+05+0.j,
       -4.18469038e+07+0.j, -2.44671993e+07+0.j, -2.64959888e+07+0.j,
       -2.72612868e+07+0.j, -1.35712751e+07+0.j, -7.86623648e+06+0.j,
       -4.47788285e+07+0.j, -3.62474580e+06+0.j, -1.31583076e+07+0.j,
       -2.71075532e+07+0.j, -1.61203224e+06+0.j, -3.29474958e+05+0.j,
       -2.34027834e+07+0.j, -2.58377672e+06+0.j, -2.34892555e+07+0.j,
       -3.20880056e+07+0.j, -1.47547772e+07+0.j, -4.19467821e+07+0.j,
       -2.77162292e+07+0.j, -3.28299966e+06+0.j, -1.37587293e+07+0.j,
       -3.43003151e+07+0.j, -2.86502876e+07+0.j, -3.53015743e+06+0.j,
       -6.42985120e+06+0.j, -3.84418078e-03+0.j, -3.66933709e+07+0.j,
       -8.89773681e+06+0.j, -4.18722231e+07+0.j, -5.41527339e+06+0.j,
       -2.45765820e+06+0.j, -3.05279198e-02+0.j, -2.97001020e+01+0.j,
       -1.45407189e+06+0.j, -1.13553558e+06+0.j, -1.12082420e+08+0.j,
       -2.60090006e-03+0.j, -5.88662419e+02+0.j, -1.80418313e+08+0.j,
       -8.68027527e-03+0.j, -5.48523271e+02+0.j, -2.60403313e+03+0.j,
       -5.50485212e-01+0.j, -2.06557742e+06+0.j, -3.66822692e+01+0.j]])

ini_t = np.array([1.55882956e-06, 3.0463594720590934e-07])
ini_typ = np.array([],dtype = str)
for i in range(2):
    if np.mod(i,3) == 0:
        ini_typ = np.append(ini_typ,"x")
    elif np.mod(i,3) == 1:
        ini_typ = np.append(ini_typ,"x")
    else:
        ini_typ = np.append(ini_typ,"x")
# ini_diag,ini_t = find_H_AC_supper(atom_number,target,ini_diag = ini_diag,ini_t = ini_t,ini_typ = ini_typ,times = 50)
# ini_diag,ini_t = find_H_AC_supper(atom_number,target,ini_diag = ini_diag,ini_t = ini_t,ini_typ = ini_typ,times = 50)
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 100, Delta = -2*np.pi*3.4*10**9, Omega = 0.2*10**6*2*np.pi)


for diag in range(len(ini_diag)):
    if ini_typ[diag] == "x":
        rotation = system.S_x
    elif ini_typ[diag] == "y":
        rotation = system.S_y
    elif ini_typ[diag] == "z":
        rotation = system.S_z
    system.psi = expm(-1j*(system.Omega*rotation + np.diag(ini_diag[diag]))*ini_t[diag])@ system.psi
# system.visualize_withwall(np.zeros(51))
Xr = system.evaluation(mode = target)[0]
print(Xr)
print(list(np.real(ini_diag)))

0.04940280365679639
[array([-3.69419234e+06, -3.22893640e+06, -5.00058320e+06, -2.79841633e+06,
       -2.36476169e+06, -8.86845772e+06, -1.16220639e+07, -7.07174941e+06,
       -2.91275808e-05, -7.87237702e+06, -1.79739730e+07, -2.01607004e+07,
       -2.05188026e+07, -1.56821854e+07, -1.09035967e+07, -1.59218550e+07,
       -2.20960351e+07, -1.16042221e+07, -6.89418543e+06, -1.82046883e+07,
       -2.77978262e-02, -4.37514085e+05, -2.02425786e+06, -7.56609966e+06,
       -2.30834440e+07, -1.31294537e+06, -5.39459229e+06, -2.58254283e+00,
       -1.60340133e+06, -7.09182453e+06, -2.60091925e+07, -2.28615718e+07,
       -9.57053560e+06, -1.27230231e+07, -3.19693008e+07, -1.61172825e+07,
       -1.34084411e+08, -2.45740907e+07, -1.57830894e+07, -6.14677630e-05,
       -1.10471061e+00, -3.39751388e+03, -7.05990219e+06, -6.99015484e-02,
       -1.39477361e+07, -2.34833249e-02, -2.54078916e-01, -4.12100584e+06,
       -1.19800925e-01, -1.36555525e+07, -1.12054889e+01]), array([-2.45192662e

In [4]:
# |W>
atom_number = 50
target = "N1N1"
fidelity_W_1 = 1e-8
t_W_1 = [3.53429767e-07]
ini_typ = ["x"]
diagonal_element_W_1 = [-1.62705976e+04-0.j, -1.03209865e+04-0.j, -11.71112435e+7-0.j,
  -1.58403566e+01-0.j, -4.05220696e+04-0.j, -6.67241820e+01-0.j,
  -5.01852541e+04-0.j, -2.13101901e+02-0.j, -1.39472807e+05-0.j,
  -6.03212194e+02-0.j, -6.60206567e+05-0.j, -2.03118900e+04-0.j,
  -3.42057633e+05-0.j, -2.04847522e+05-0.j, -8.06182838e+05-0.j,
  -7.00379868e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j]

print(list(np.real(diagonal_element_W_1)))

system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 50, Delta = -2*np.pi*3.4*10**9, Omega = 0.2*10**6*2*np.pi)

for diag in range(1):
    if ini_typ[diag] == "x":
        rotation = system.S_x
    elif ini_typ[diag] == "y":
        rotation = system.S_y
    elif ini_typ[diag] == "z":
        rotation = system.S_z
    system.psi = expm(-1j*(system.Omega*rotation + np.diag(diagonal_element_W_1))*t_W_1[diag])@ system.psi
# system.visualize_withwall(diagonal_element_W_1)
target = "N1N1"
Xr = system.evaluation(mode = target)[0]
print((1-Xr)/2)


delta0 =  np.array([0.])
order0 =  np.array([28.96868897])
t0 =  1.5155999580658109e-07
typ0 = "x"
delta1 =  np.array([-81031001.57593244])
order1 =  np.array([47.19472869])
t1 =  2.3468752534542159e-07
typ1 = "x"
Delta =  -5946171681.087355
atom_number = 50
ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))
# ini_sequence = sequence(np.array([photons("x",np.array([13]),np.array([-133.8*10**(6)*0]),1.054*10**(-6)),photons("x",np.array([13]),np.array([-133.8*10**(6)]),0.496*10**(-6))]))

system = cavity_atom_light_system(atom_number = 50, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
rotation = system.S_x
# system.psi = expm(-1j*(system.Omega*rotation + np.diag(diagonal_element_GHZ_1))*(t0+t1))@ system.psi
system.evolve_photons(ini_sequence)
# system.visualize()
# system.show_occupation()

H_AC_1p = system.show_H_AC_photons(ini_sequence)[1]

fig,ax = plt.subplots()
# ax.scatter(np.arange(51),np.real(diagonal_element_GHZ_1))
ax.scatter(np.arange(51),np.real(H_AC_1p)*10**(-6))
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/MHz")
ax.set_title("Energy level, One Light Generating W state")
plt.show()

target = "N1N1"

print(system.evaluation(mode = target))
# system.visualize_withwall(H_AC_1p)
print(list(np.real(H_AC_1p)))

[-16270.5976, -10320.9865, -117111243.5, -15.8403566, -40522.0696, -66.724182, -50185.2541, -213.101901, -139472.807, -603.212194, -660206.567, -20311.89, -342057.633, -204847.522, -806182.838, -700379.868, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94, -682935.94]
0.9903501271847687
(-0.9200017631770638, 0)
[-3055864.2873986405, -6968082.146935489, -26567195.60147591, -68512190.80420399, -12534491.498873688, -4182804.3626613137, -2004429.26184368, -1154392.366017346, -741377.8307805437, -511489.08908942866, -371162.3764264518, -279598.8152834317, -216763.59545395573, -171907.9987114512, -138856.09612265936, -113859.78860134524, -9

In [13]:
# |25>
target = "N25N25"
atom_number = 50
ini_diag = np.array([[-5.56799319e+06-0.j, -5.16530005e+06-0.j, -4.21595406e+06-0.j,
  -3.20339303e+06-0.j, -2.60645477e+06-0.j, -2.49247062e+06-0.j,
  -2.63349926e+06-0.j, -2.64403201e+06-0.j, -2.27133117e+06-0.j,
  -1.47589768e+06-0.j, -4.23676229e+05-0.j, -2.81531417e-01-0.j,
  -2.09019415e+05-0.j, -1.15545093e-05-0.j, -9.18734232e+02-0.j,
  -1.70593708e+06-0.j, -3.91829080e+05-0.j, -6.79208738e+06-0.j,
  -9.88569106e+06-0.j, -1.15102955e+05-0.j, -1.78599540e+07-0.j,
  -1.71836479e-06-0.j, -3.21190414e-06-0.j, -1.29041712e+07-0.j,
  -3.15671102e+07-0.j, -2.65693048e+07-0.j, -2.47318641e+07-0.j,
  -2.36066660e+07-0.j, -1.81628398e+07-0.j, -7.35897993e-03-0.j,
  -7.84828011e+07-0.j, -5.43877563e-04-0.j, -7.44975523e+07-0.j,
  -8.52306150e-02-0.j, -1.15139490e+07-0.j, -2.72486035e+08-0.j,
  -5.02604226e+02-0.j, -5.92354536e+08-0.j, -1.80327741e+04-0.j,
  -6.48973319e+02-0.j, -6.81282316e+05-0.j, -1.07307512e+03-0.j,
  -1.55516669e+06-0.j, -1.70335446e+04-0.j, -4.90200171e+06-0.j,
  -1.11371055e+05-0.j, -3.01348744e+06-0.j, -9.49245757e+05-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j],
 [-7.04675320e+05-0.j, -9.23607171e+03-0.j, -1.12054257e+08-0.j,
  -2.66498218e+02-0.j, -8.19990613e+06-0.j, -2.83638392e+07-0.j,
  -2.36968946e-04-0.j, -4.20381842e+07-0.j, -7.84287578e-05-0.j,
  -1.02473199e+07-0.j, -9.26096994e-02-0.j, -8.28881175e+06-0.j,
  -9.41653209e+06-0.j, -1.93756860e+07-0.j, -1.62990488e+07-0.j,
  -2.54776753e+07-0.j, -1.29462793e+07-0.j, -2.48667178e+07-0.j,
  -2.08137080e+07-0.j, -1.86505896e+07-0.j, -3.07304457e+07-0.j,
  -2.30180312e+07-0.j, -3.24040662e+06-0.j, -5.95808520e+05-0.j,
  -2.09574210e-06-0.j, -1.51091656e+07-0.j, -3.75189657e-06-0.j,
  -1.85745965e+07-0.j, -6.93378400e+05-0.j, -1.43324568e+08-0.j,
  -2.17825751e+04-0.j, -1.10304699e+07-0.j, -9.77324559e+08-0.j,
  -4.88639030e-04-0.j, -8.47468401e-02-0.j, -8.58280546e+06-0.j,
  -3.95145506e-02-0.j, -6.76046600e+07-0.j, -5.73834850e-01-0.j,
  -1.25916977e+07-0.j, -5.24297529e+00-0.j, -2.37166942e+07-0.j,
  -7.29504408e+02-0.j, -2.34009741e+03-0.j, -7.74391586e+04-0.j,
  -3.46143107e+04-0.j, -4.94697759e+06-0.j, -8.11165777e+05-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j],
 [-1.00000000e+06-0.j, -1.66311631e+06-0.j, -4.70077470e+05-0.j,
  -1.96212600e+05-0.j, -3.96258497e+07-0.j, -5.33174257e+08-0.j,
  -1.18658847e+03-0.j, -1.61062138e+02-0.j, -8.06114535e-01-0.j,
  -2.75696220e+07-0.j, -1.87759297e-01-0.j, -1.30283931e+04-0.j,
  -1.48907779e+08-0.j, -8.68855531e+06-0.j, -1.76890608e+07-0.j,
  -8.14444159e+07-0.j, -4.56972544e-05-0.j, -2.34734711e+07-0.j,
  -2.84455856e-05-0.j, -6.17490048e+07-0.j, -5.04490206e+06-0.j,
  -7.66810405e+06-0.j, -8.89815276e+06-0.j, -3.47076682e+07-0.j,
  -5.47546679e+06-0.j, -2.42851232e+07-0.j, -1.78937734e+07-0.j,
  -8.23342018e-07-0.j, -4.43746523e+07-0.j, -1.59699358e+07-0.j,
  -1.18329558e+08-0.j, -1.65152483e+07-0.j, -2.42622707e+08-0.j,
  -8.30196473e-03-0.j, -2.50262388e+07-0.j, -1.84691161e-03-0.j,
  -2.09346670e+07-0.j, -2.54840268e-01-0.j, -2.42496584e+07-0.j,
  -2.11588835e+07-0.j, -1.78083828e+02-0.j, -7.46155725e+07-0.j,
  -7.89632163e+04-0.j, -1.48310905e+04-0.j, -6.69880380e+06-0.j,
  -4.70888866e+05-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j]])
ini_t = np.array([1.03181688e-06, 3.17979239e-07, 2.58272342e-07])
ini_typ = np.array([],dtype = str)
for i in range(3):
    if np.mod(i,3) == 0:
        ini_typ = np.append(ini_typ,"x")
    elif np.mod(i,3) == 1:
        ini_typ = np.append(ini_typ,"x")
    else:
        ini_typ = np.append(ini_typ,"x")
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 100, Delta = -2*np.pi*3.4*10**9, Omega = 0.2*10**6*2*np.pi)

for diag in range(len(ini_diag)):
    if ini_typ[diag] == "x":
        rotation = system.S_x
    elif ini_typ[diag] == "y":
        rotation = system.S_y
    elif ini_typ[diag] == "z":
        rotation = system.S_z
    system.psi = expm(-1j*(system.Omega*rotation + np.diag(ini_diag[diag]))*ini_t[diag])@ system.psi

# system.visualize_withwall(np.zeros(51))
    
Xr = system.evaluation(mode = target)[0]
print((1-Xr)/2)

print(list(np.real(ini_diag)))

0.9921164925593445
[array([-5.56799319e+06, -5.16530005e+06, -4.21595406e+06, -3.20339303e+06,
       -2.60645477e+06, -2.49247062e+06, -2.63349926e+06, -2.64403201e+06,
       -2.27133117e+06, -1.47589768e+06, -4.23676229e+05, -2.81531417e-01,
       -2.09019415e+05, -1.15545093e-05, -9.18734232e+02, -1.70593708e+06,
       -3.91829080e+05, -6.79208738e+06, -9.88569106e+06, -1.15102955e+05,
       -1.78599540e+07, -1.71836479e-06, -3.21190414e-06, -1.29041712e+07,
       -3.15671102e+07, -2.65693048e+07, -2.47318641e+07, -2.36066660e+07,
       -1.81628398e+07, -7.35897993e-03, -7.84828011e+07, -5.43877563e-04,
       -7.44975523e+07, -8.52306150e-02, -1.15139490e+07, -2.72486035e+08,
       -5.02604226e+02, -5.92354536e+08, -1.80327741e+04, -6.48973319e+02,
       -6.81282316e+05, -1.07307512e+03, -1.55516669e+06, -1.70335446e+04,
       -4.90200171e+06, -1.11371055e+05, -3.01348744e+06, -9.49245757e+05,
       -1.00000000e+06, -1.00000000e+06, -1.00000000e+06]), array([-7.04675320e+

In [596]:
# SSS Illustration
delta0 =  np.array([0.0])
order0 =  np.array([13.37773625])
t0 =  1.066245874759036e-06
typ0 = "x"
delta1 =  np.array([-2.43289333e+08])
order1 =  np.array([13.82725477])
t1 =  4.97960779192943e-07
typ1 = "x"
Delta =  -4115000000.0
atom_number = 50
ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))
target = "SSSN"


system = cavity_atom_light_system(atom_number = 50, cooperativity = 200, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
system.visualize_withwall(np.zeros(51))
system.evolve_photons_t(ini_sequence,t0)
system.visualize_withwall(np.zeros(51))
system.evolve_photons_t(ini_sequence,t0+0.3*t1)
system.visualize_withwall(diag)
system.evolve_photons_t(ini_sequence,t0+t1)
system.visualize_withwall(diag)

In [4]:
delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
t0 =  4.1399943765870936e-07
typ0 = "x"
delta1 =  np.array([-1770840.01367247])
order1 =  np.array([32.74001811149749])
t1 =  2.1773138739050565e-06
typ1 = "x"
Delta =  -2*np.pi*3.4*10**9

ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

atom_number = 50

animate_system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)

# animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 56, Delta = -197*Gamma, Omega = 0.2*10**6*2*np.pi)
# animate_para1 = [0.0e-06,0.0e-06,1.593302185297957e-06,74,-423699254.15045,0.0e-06]
# animate_para2 = [0.0e-06,0.0e-06,0.0e-06,74,-423699254.15045,2.4134381601596233e-06]

# 时间步长
time_step = 0.2*10**(-7)

num_frames = int((t0+t1)/time_step+1)

# facecolors の　list
colors = []
states = []

# 更新frames——
for frame in range(num_frames):
    t = frame*time_step
    colors.append(animate_system.evolve_photons_t(ini_sequence,t))
    states.append(animate_system.psi)
    animate_system.system_reset()
    print("\r", end="")
    print(f"frame generated:{frame}/{num_frames}", end="")
    sys.stdout.flush()





frame generated:129/130

In [5]:
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, np.pi*4/2, 100)
theta_grid, phi_grid = np.meshgrid(theta, phi)
y = np.sin(theta_grid) * np.cos(phi_grid)
x = -np.sin(theta_grid) * np.sin(phi_grid)
z = -np.cos(theta_grid)

# 生成动图基础图片：
image_list = []
for i in range(num_frames):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    q_func = colors[i]
    norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1,cmap=cm.coolwarm, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Husimi-Q Representation')

    # 存图
    plt.savefig(f"fig/20231121_1stepGHZ{i}.jpg")
    plt.close()
    image_list.append(f"fig/20231121_1stepGHZ{i}.jpg")
    print("\r", end="")
    print(f"data generated:{i}/{num_frames}", end="")
    sys.stdout.flush()
    
# 搞动图：
gif_name = "fig/20231121_1stepGHZ.gif"
duration = 0.05
create_gif(image_list, gif_name,duration)

data generated:129/130